In [2]:
import sys
sys.path.insert(0,'..')

from light_pipe_geo import processing, pipeline, concurrency, mercantile, gridding

# Get Tiles From a GeoTiff File

In [3]:
from osgeo import gdal
import numpy as np

gdal.UseExceptions()

savepath = "../data/examples/make_tiles/tiles.tif"
tif_path = "../data/image.tif"

lp_sample = processing.LightPipeSample(tif_path)

tiles = lp_sample.tile()
preds = list()
for tile in tiles:
    preds += [np.any(tile.X)]
lp_sample.save(savepath, preds)


# Create Label Mask from `.shp` File

In [4]:
inputs = [
    {
        'dataset': '../data/image.tif',
        'datasources': ['../data/label/label.shp']
    },
]
pipe = pipeline.LightPipeline(inputs)

for sample in pipe:
    tiles = sample.tile()
    preds = list()
    for tile in tiles:
        preds += [np.any(tile.y)]

    savepath = f"../data/examples/make_masks/shp_sample_{sample.data.uid}"
    sample.save(savepath, preds) 

# Create Label Mask from `.geojson` File

In [5]:
inputs = [
    {
        'dataset': '../data/image.tif',
        'datasources': ['../data/label.geojson']
    },
]
pipe = pipeline.LightPipeline(inputs)

for sample in pipe:
    tiles = sample.tile()
    preds = list()
    for tile in tiles:
        preds += [np.any(tile.y)]

    savepath = f"../data/examples/make_masks/geojson_sample_{sample.data.uid}"
    sample.save(savepath, preds) 

# Create Label Mask from `.geojson` File With Different CRS

In [6]:
inputs = [
    {
        'dataset': '../data/image.tif',
        'datasources': ['../data/label_utm.geojson']
    },
]
pipe = pipeline.LightPipeline(inputs)

for sample in pipe:
    tiles = sample.tile()
    preds = list()
    for tile in tiles:
        preds += [np.any(tile.y)]

    savepath = f"../data/examples/make_masks/utm_geojson_sample_{sample.data.uid}"
    sample.save(savepath, preds) 

# Create Grid Cell Datasets

In [7]:
inputs = [
    {
        'datum': '../data/image.tif',
        'is_label': False,
    },
    {
        'datum': '../data/label/label.shp',
        'is_label': True
    },
    {
        'datum': '../data/label.geojson',
        'is_label': True
    },    
    {
        'datum': '../data/label_utm.geojson',
        'is_label': True
    }    
]
gh = processing.GridSampleMaker()
pipe = pipeline.LightPipeline(inputs, processors=[gh])

for sample in pipe:
    tiles = sample.tile()
    preds = list()
    for tile in tiles:
        preds += [np.any(tile.y)]

    savepath = f"../data/examples/make_grid_cells/grid_cell_{sample.data.uid}.tif"
    sample.save(savepath, preds) 